In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/RL_real_0627_ind_index_value_weighted

/content/drive/.shortcut-targets-by-id/1w6OSQ_9ZH9r3830ldxRe1TGWfVlxxXWo/RL_real_0625_ind_index_value_weighted


In [ ]:
import pandas as pd
import numpy as np

path = '10_Industry_Portfolios_Daily.csv'


start_row = 10


n_data_rows = 25995 - start_row


raw_header = pd.read_csv(
    path,
    skiprows=start_row - 1,
    nrows=1,
    header=None,
    engine='python'
)


df = pd.read_csv(
    path,
    skiprows=start_row,
    nrows=n_data_rows,
    header=None,
    names=raw_header.iloc[0].tolist(),
    engine='python'
)


first_col = df.columns[0]
df[first_col] = pd.to_datetime(df[first_col], format='%Y%m%d', errors='coerce')
df = df.rename(columns={first_col: 'Date'})

for col in df.columns:
    if col != 'Date':
        df[col] = pd.to_numeric(df[col].astype(str).str.strip(), errors='coerce')

# 5) Melt to daily panel
panel_daily = df.melt(
    id_vars='Date',
    var_name='Industry',
    value_name='Value'
).dropna(subset=['Value'])


# Monthly averages
panel_monthly = (
    panel_daily
    .set_index('Date')
    .groupby(['Industry', pd.Grouper(freq='M')])['Value']
    .apply(lambda x: np.prod(1 + x / 100) - 1 ) # Assuming 'Value' is a percentage return
    .reset_index(name='Monthly_Return') # Changed column name to reflect return
)


# (Optional) set MultiIndex
panel_monthly  = panel_monthly .set_index(['Date','Industry']).sort_index()

panel_monthly['Next_Monthly_Return'] = (
    panel_monthly.reset_index() # Reset index to groupby by 'Industry'
    .groupby('Industry')['Monthly_Return']
    .shift(-1)
    .values # Get the values to assign back to the DataFrame
)


panel_monthly.to_csv('savings/industrial_monthly_data_vw.csv')




/tmp/ipython-input-3-2893595098.py:52: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .groupby(['Industry', pd.Grouper(freq='M')])['Value']
